# Оценка качества моделей на смержинных текстах squad dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Evaluation function is based on 
https://colab.research.google.com/github/fastforwardlabs/ff14_blog/blob/master/_notebooks/2020-06-09-Evaluating_BERT_on_SQuAD.ipynb#scrollTo=qBGB1LPFBQ8z

In [2]:
!pip install import-ipynb
%cd '/content/drive/My Drive/QA_training/notebooks'
import import_ipynb
from my_functions import *

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=02fb2b62aa81191878cbbd9e3e45d68f523f433f7a7cd30b7f6d13c1a5a819f9
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
/content/drive/My Drive/QA_training/notebooks
importing Jupyter notebook from my_functions.ipynb
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-fvy5hito
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-fvy5hito
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9MB 9.1MB/s 
     |████████████████████████████████| 890kB 44.3MB/s 
  Created wheel for transformers: filename=transformers-4.3.0.dev0-cp36-none-any.whl size=1735861 sha256=065ed7627b6217f106442bb6afd1df2469d2aac465fa6c7523bee793db5e1bff
  Store

In [ ]:
dataset_names = ['initial dataset', 'same topic dataset', 'different topics dataset']
dataset_paths = ['/content/drive/MyDrive/QA_training/merged datasets/squad/initial_dev.csv',
             '/content/drive/MyDrive/QA_training/merged datasets/squad/merged_by_2_dev.csv',
             '/content/drive/MyDrive/QA_training/merged datasets/squad/random_merged_by_2_dev.csv']
             
#datasets = [prepare_and_merge_data(csv_paths[i], tokenizer,data_type ='squad', text_num=i+1) for i in range(3)]

In [ ]:
model_names = ['bert', 'same topic bert', 'different topics bert']
model_paths = ['/content/drive/MyDrive/QA_training/models/squad_models/merged_by_1_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/merged_by_2_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/randon_merged_by_2_bert']
model_5_epoch_paths = ['/content/drive/MyDrive/QA_training/models/squad_models/merg_by_1_5_epochs',
                       '/content/drive/MyDrive/QA_training/models/squad_models/merg_by_2_5_epochs',
                       '/content/drive/MyDrive/QA_training/models/squad_models/random_merg_by_2_5_epochs']


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Сравнение и результаты

## Модели, обученные на 3 эпохах:

In [ ]:
table_f1 = np.zeros((3, 3), dtype=object)
for model_idx in range(len(model_paths)):
    model =  BertForQuestionAnswering.from_pretrained(model_paths[model_idx])
    trainer = Trainer(
                model=model,                         # the instantiated 🤗 Transformers model to be trained
            )
    
    for data_idx in range(len(dataset_paths)):
        print(f"Step {model_idx * len(dataset_paths) + data_idx +1} of {len(dataset_paths)*len(model_paths)}")
        
        if data_idx == 0:
            text_num = 1
        else:
            text_num = 2

        val_dataset, answers = prepare_and_merge_data(dataset_paths[data_idx], tokenizer, data_type ='squad', text_num=text_num, return_answers=True)
    
        results = evaluate_score_squad(trainer, answers, tokenizer, eval_dataset = val_dataset)
        del val_dataset


        table_f1[model_idx, data_idx] = results['f1_mean']

        np.save('/content/drive/MyDrive/QA_training/results/squaq_merged/table_f1', table_f1)

Step 1 of 9


Step 2 of 9
Step 3 of 9
Step 4 of 9


Step 5 of 9
Step 6 of 9
Step 7 of 9


Step 8 of 9
Step 9 of 9


In [ ]:
df = pd.DataFrame(data=table_f1, index=model_names, columns=dataset_names)
df

,initial dataset,same topic dataset,different topics dataset
bert,0.690437,0.677931,0.673141
same topic bert,0.644689,0.638234,0.653796
different topics bert,0.693921,0.682816,0.683464


In [ ]:
df.to_csv('/content/drive/MyDrive/QA_training/results/squaq_merged/table_f1.csv')


## Модели, обученные на 5 эпохах:

In [ ]:
table_f1_5e = np.zeros((3, 3), dtype=object)
for model_idx in range(len(model_5_epoch_paths)):
    model =  BertForQuestionAnswering.from_pretrained(model_5_epoch_paths[model_idx])
    trainer = Trainer(
                model=model,                         # the instantiated 🤗 Transformers model to be trained
            )
    
    for data_idx in range(len(dataset_paths)):
        print(f"Step {model_idx * len(dataset_paths) + data_idx +1} of {len(dataset_paths)*len(model_5_epoch_paths)}")
        print("Imported model:", model_5_epoch_paths[model_idx])
        print("Imported dataset:", dataset_paths[data_idx])

        if data_idx == 0:
            text_num = 1
        else:
            text_num = 2

        val_dataset, answers = prepare_and_merge_data(dataset_paths[data_idx], tokenizer, data_type ='squad', text_num=text_num, return_answers=True)
    
        results = evaluate_score_squad(trainer, answers, tokenizer, eval_dataset = val_dataset)
        del val_dataset


        table_f1_5e[model_idx, data_idx] = results['f1_mean']

        np.save('/content/drive/MyDrive/QA_training/results/squaq_merged/table_f1_5e', table_f1_5e)

Step 1 of 9
Imported model: /content/drive/MyDrive/QA_training/models/squad_models/merg_by_1_5_epochs
Imported dataset: /content/drive/MyDrive/QA_training/merged datasets/squad/initial_dev.csv


Step 2 of 9
Imported model: /content/drive/MyDrive/QA_training/models/squad_models/merg_by_1_5_epochs
Imported dataset: /content/drive/MyDrive/QA_training/merged datasets/squad/merged_by_2_dev.csv
Step 3 of 9
Imported model: /content/drive/MyDrive/QA_training/models/squad_models/merg_by_1_5_epochs
Imported dataset: /content/drive/MyDrive/QA_training/merged datasets/squad/random_merged_by_2_dev.csv
Step 4 of 9
Imported model: /content/drive/MyDrive/QA_training/models/squad_models/merg_by_2_5_epochs
Imported dataset: /content/drive/MyDrive/QA_training/merged datasets/squad/initial_dev.csv


Step 5 of 9
Imported model: /content/drive/MyDrive/QA_training/models/squad_models/merg_by_2_5_epochs
Imported dataset: /content/drive/MyDrive/QA_training/merged datasets/squad/merged_by_2_dev.csv
Step 6 of 9
Imported model: /content/drive/MyDrive/QA_training/models/squad_models/merg_by_2_5_epochs
Imported dataset: /content/drive/MyDrive/QA_training/merged datasets/squad/random_merged_by_2_dev.csv
Step 7 of 9
Imported model: /content/drive/MyDrive/QA_training/models/squad_models/random_merg_by_2_5_epochs
Imported dataset: /content/drive/MyDrive/QA_training/merged datasets/squad/initial_dev.csv


Step 8 of 9
Imported model: /content/drive/MyDrive/QA_training/models/squad_models/random_merg_by_2_5_epochs
Imported dataset: /content/drive/MyDrive/QA_training/merged datasets/squad/merged_by_2_dev.csv
Step 9 of 9
Imported model: /content/drive/MyDrive/QA_training/models/squad_models/random_merg_by_2_5_epochs
Imported dataset: /content/drive/MyDrive/QA_training/merged datasets/squad/random_merged_by_2_dev.csv


In [ ]:
table_f1_5e = np.load('/content/drive/MyDrive/QA_training/results/squaq_merged/table_f1_5e.npy', allow_pickle=True)

In [ ]:
df = pd.DataFrame(data=table_f1_5e, index=model_names, columns=dataset_names)
df

,initial dataset,same topic dataset,different topics dataset
bert,0.684866,0.665622,0.664877
same topic bert,0.645934,0.637256,0.653131
different topics bert,0.67984,0.669458,0.672936


In [ ]:
df.to_csv('/content/drive/MyDrive/QA_training/results/squaq_merged/table_f1_5e.csv')

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
val_csv_path = '/content/drive/MyDrive/QA_training/merged datasets/squad/initial_dev.csv'

val_dataset = prepare_and_merge_data(val_csv_path, tokenizer, text_num=1, parts_num = 1, part = 1)

In [ ]:
dir = "/content/drive/MyDrive/QA_training/models/squad_models/merged_by_1_bert"

model =  BertForQuestionAnswering.from_pretrained(dir)

In [ ]:
training_args = TrainingArguments(
    output_dir=dir,          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    logging_dir=dir+'/logs',            # directory for storing logs
    logging_steps=250,
    prediction_loss_only=False,
    label_names = ["start_positions", "end_positions"],
    overwrite_output_dir = True
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    #args=training_args,                  # training arguments, defined above
    #train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset

)

In [ ]:
answers = pd.read_csv(val_csv_path).answer

In [ ]:
f_1 = evaluate_score_squad(trainer, answers, tokenizer)

In [ ]:
f_1

{'f1_mean': 0.6904371213089024}

Сделали все необходимые функции, теперь надо определиться с порядком действий:

1. Берем валидационный датасет и прогоняем на нем все модели и смотрим f1 score

2. Берем валидационный датасет из трех параграфов и прогоняем на нем:
    
    2.1 Для модели, обученной на одинарных текстах, мы прогоняем ее на датасетах по одному параграфу (надо сделать функцию, которая берет эти параграфы из  общего датасета) 
    2.2 Далее для каждый из 3 результатов прогонки мы трансформируем в текст и сохроняем
    2.3 Сравниваем результат для каждого параграфа, если все результаты показали пустой ответ, то конечный ответ будет тоже пустым, если в хоть одном параграфе будет непустой ответ, то выбираем тот, у которого наибольший score.
    2.4 повторяем тоже самое на второй модели с 2 параграфами


Время: пункт 1: 3 прогонки (+2) = 3(5). Пункт 2: 3+2 (+3+2) = 5(10)

Всего: 8 (15) = час (два часа)








Пункт 1.

In [ ]:
model_names = ['bert_1_text', 'bert_2_texts', 'bert_3_texts', 'bert_2_random_texts', 'bert_3_rendom_texts']
model_paths = ['/content/drive/MyDrive/QA_training/models/squad_models/merged_by_1_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/merged_by_2_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/merged_by_3_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/randon_merged_by_2_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/randon_merged_by_3_bert']

In [ ]:
dataset_path = '/content/drive/MyDrive/QA_training/merged datasets/squad/initial_dev.csv'

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
val_dataset = prepare_and_merge_data(dataset_path, tokenizer)
answers = pd.read_csv(dataset_path).answer

f1_mean = np.zeros(len(model_paths))

for i in range(len(model_paths)):
    print(f"Step {i+1} of {len(model_paths)}.")
    dir = model_paths[i]
    model =  BertForQuestionAnswering.from_pretrained(dir)
    
    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        #args=training_args,                  # training arguments, defined above
        #train_dataset=train_dataset,         # training dataset
        eval_dataset=val_dataset,            # evaluation dataset
    )

    f1_mean[i] = evaluate_score_squad(trainer, answers, tokenizer)['f1_mean']

    np.save('/content/drive/MyDrive/QA_training/results/text_prep_results/f1_for_different_models_and_1_text_dev',
            f1_mean)


Step 1 of 5.


KeyboardInterrupt: ignored

In [ ]:
df_f1 = pd.DataFrame(data=f1_mean.reshape(-1, 1), index = model_names , columns='f1')
df_f1.to_csv('/content/drive/MyDrive/QA_training/results/text_prep_results/f1_for_different_models_and_1_text_dev.csv')

Пункт 2:

надо сделать функцию, которая сравнивает несколько ответов и выбирает нужный

SyntaxError: ignored

In [ ]:
model_names = ['bert_1_text', 'bert_2_texts', 'bert_3_texts', 'bert_2_random_texts', 'bert_3_rendom_texts']
model_paths = ['/content/drive/MyDrive/QA_training/models/squad_models/merged_by_1_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/merged_by_2_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/merged_by_3_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/randon_merged_by_2_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/randon_merged_by_3_bert']

In [ ]:
dataset_names = ['merged_by_3', 'random_merged_by_3']
dataset_paths = ['/content/drive/MyDrive/QA_training/merged datasets/squad/merged_by_3_dev.csv',
                 '/content/drive/MyDrive/QA_training/merged datasets/squad/random_merged_by_3_dev.csv']

Прогоним для модели bert_1_text:

Добавил сюда костыль - функция prepare_and_merge_eval_data которая подставляет только пустые ответы в датасет, на проверку не должна влиять, если брать ответы и датафрейма



In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
#val_dataset = prepare_and_merge_data(dataset_path, tokenizer)
#answers = pd.read_csv(dataset_path).answer

dir = model_paths[0]
model =  BertForQuestionAnswering.from_pretrained(dir)

pred_texts = []
for i in range(3):
    print(f"Step: {i}")
    val_dataset = prepare_and_merge_eval_data(dataset_paths[0], tokenizer, text_num=3, text_ids = [i+1])

    trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    #args=training_args,                  # training arguments, defined above
    #train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    )

    predictions = trainer.predict(val_dataset)
    pred_texts.append(return_text_results(predictions, val_dataset, tokenizer))
    np.save('/content/drive/MyDrive/QA_training/results/text_prep_results/3_answers_for_bert', pred_texts)


Step: 0


Step: 1


Step: 2


Теперь надо выбрать один ответ среди трех для каждого вопроса:

In [ ]:
def select_correct_answer(answers):
    answers = np.array(answers)

    final_answers = []
    for i in range(len(answers[0])):
        texts_and_scores = answers[:, i]
        no_empty_texts = []
        for text, score in texts_and_scores:
            if text != '':
              no_empty_texts.append((text, score))
        if no_empty_texts:
            final_answers.append(sorted(no_empty_texts, key=lambda x: float(x[1]), reverse=True)[0][0])
        else:
            final_answers.append('')
    return final_answers


In [ ]:
correct_pred = select_correct_answer(pred_texts)

answers = pd.read_csv(dataset_paths[0]).answer


f1 = []
for pred_text, answer in zip(correct_pred, answers):
    if type(answer) != str:
        answer = ''
    f1.append(compute_f1(pred_text, answer))

f1_mean = np.mean(f1)

In [ ]:
bert_1_text_f1 = f1_mean
bert_1_text_f1

0.649740176346341

Прогоним для модели bert_2_texts:

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
#val_dataset = prepare_and_merge_data(dataset_path, tokenizer)
#answers = pd.read_csv(dataset_path).answer

dir = model_paths[1]
model =  BertForQuestionAnswering.from_pretrained(dir)

pred_texts_2 = []
for i in range(2):
    print(f"Step: {i+1}")
    val_dataset = prepare_and_merge_eval_data(dataset_paths[0], tokenizer, text_num=3, text_ids = [i+1, i+2])

    trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    #args=training_args,                  # training arguments, defined above
    #train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    )

    predictions = trainer.predict(val_dataset)
    pred_texts_2.append(return_text_results(predictions, val_dataset, tokenizer))
    np.save('/content/drive/MyDrive/QA_training/results/text_prep_results/2_answers_for_bert_2', pred_texts_2)


Step: 1


Step: 2


In [ ]:
correct_pred = select_correct_answer(pred_texts_2)

answers = pd.read_csv(dataset_paths[0]).answer


f1 = []
for pred_text, answer in zip(correct_pred, answers):
    if type(answer) != str:
        answer = ''
    f1.append(compute_f1(pred_text, answer))

f1_mean = np.mean(f1)
f1_mean

0.6166597069093449

Прогоним модель, обученную на трех текстах:

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
val_dataset = prepare_and_merge_data(dataset_paths[0], tokenizer, text_num=3)
answers = pd.read_csv(dataset_paths[0]).answer

dir = model_paths[2]
model =  BertForQuestionAnswering.from_pretrained(dir)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    #args=training_args,                  # training arguments, defined above
    #train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    )

f_1 = evaluate_score_squad(trainer, answers, tokenizer)

In [ ]:
f_1

{'f1_mean': 0.5973775186654062}

Прогоним для модели, обученной на 2 кусках текста, один из которых рандомный

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
#val_dataset = prepare_and_merge_data(dataset_path, tokenizer)
#answers = pd.read_csv(dataset_path).answer

dir = model_paths[3]
model =  BertForQuestionAnswering.from_pretrained(dir)

pred_texts_2 = []
for i in range(2):
    print(f"Step: {i+1}")
    val_dataset = prepare_and_merge_eval_data(dataset_paths[0], tokenizer, text_num=3, text_ids = [i+1, i+2])

    trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    #args=training_args,                  # training arguments, defined above
    #train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    )

    predictions = trainer.predict(val_dataset)
    pred_texts_2.append(return_text_results(predictions, val_dataset, tokenizer))
    np.save('/content/drive/MyDrive/QA_training/results/text_prep_results/random_2_answers_for_bert_2', pred_texts_2)


Step: 1


Step: 2


In [ ]:
correct_pred = select_correct_answer(pred_texts_2)

answers = pd.read_csv(dataset_paths[0]).answer


f1 = []
for pred_text, answer in zip(correct_pred, answers):
    if type(answer) != str:
        answer = ''
    f1.append(compute_f1(pred_text, answer))

f1_mean = np.mean(f1)
f1_mean

0.664671157957805

Прогоним на модели, обученной на трех текстах, два из которых рандомные

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
val_dataset = prepare_and_merge_data(dataset_paths[0], tokenizer, text_num=3)
answers = pd.read_csv(dataset_paths[0]).answer

dir = model_paths[4]
model =  BertForQuestionAnswering.from_pretrained(dir)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    #args=training_args,                  # training arguments, defined above
    #train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    )

f_1 = evaluate_score_squad(trainer, answers, tokenizer)
f_1

{'f1_mean': 0.6572493453439371}

In [ ]:
df = pd.read_csv(dataset_paths[0])

In [ ]:
df

,question,t1,t2,t3,text_number,answer
0,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...","The English name ""Normans"" comes from the Fren...",1,France
1,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...","The English name ""Normans"" comes from the Fren...",1,10th and 11th centuries
2,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...","The English name ""Normans"" comes from the Fren...",1,"Denmark, Iceland and Norway"
3,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...","The English name ""Normans"" comes from the Fren...",1,Rollo
4,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...","The English name ""Normans"" comes from the Fren...",1,10th century
...,...,...,...,...,...,...
11868,What is the seldom used force unit equal to on...,"For certain physical scenarios, it is impossib...",The connection between macroscopic nonconserva...,"The pound-force has a metric counterpart, less...",3,sthène
11869,What does not have a metric counterpart?,"For certain physical scenarios, it is impossib...",The connection between macroscopic nonconserva...,"The pound-force has a metric counterpart, less...",3,NaN
11870,What is the force exerted by standard gravity ...,"For certain physical scenarios, it is impossib...",The connection between macroscopic nonconserva...,"The pound-force has a metric counterpart, less...",3,NaN
11871,What force leads to a commonly used unit of mass?,"For certain physical scenarios, it is impossib...",The connection between macroscopic nonconserva...,"The pound-force has a metric counterpart, less...",3,NaN


Сравним результаты

In [ ]:
df_f1['f1_for_3_texts'] = [bert_1_text_f1, 0.6166597069093449, 0.5973775186654062, 0.664671157957805, 0.6572493453439371]


In [ ]:
df_f1.columns = ['f1_for_1_text', 'f1_for_3_texts']


In [ ]:
df_f1


In [ ]:
df_f1.to_csv('/content/drive/MyDrive/QA_training/results/text_prep_results/f1_for_different_models.csv')


Сривним результаты разных моделей на обычном датасете в двух случаях есть овтет или нет ответа:

In [ ]:
model_names = ['bert_1_text', 'bert_2_texts', 'bert_3_texts', 'bert_2_random_texts', 'bert_3_rendom_texts']
model_paths = ['/content/drive/MyDrive/QA_training/models/squad_models/merged_by_1_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/merged_by_2_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/merged_by_3_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/randon_merged_by_2_bert',
               '/content/drive/MyDrive/QA_training/models/squad_models/randon_merged_by_3_bert']
l_m = len(model_paths)

In [ ]:
dataset_paths = ['/content/drive/MyDrive/QA_training/merged datasets/squad/initial_dev_f/with_answer.csv',
                  '/content/drive/MyDrive/QA_training/merged datasets/squad/initial_dev_f/no_answer.csv']
l_d = len(dataset_paths)

In [ ]:
f1_mean = np.zeros((l_m, l_d))

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


for j in range(l_m):
        dir = model_paths[j]
        model =  BertForQuestionAnswering.from_pretrained(dir)
        trainer = Trainer(
            model=model,                         # the instantiated 🤗 Transformers model to be trained
            #args=training_args,                  # training arguments, defined above
            #train_dataset=train_dataset,         # training dataset
            #eval_dataset=val_dataset,            # evaluation dataset
        )
        for i in range(l_d):
            print(f"Step {i+j+1} of {l_m*l_d}.")
            val_dataset = prepare_and_merge_data(dataset_paths[i], tokenizer)
            answers = pd.read_csv(dataset_paths[i]).answer
            f1_mean[j, i] = evaluate_score_squad(trainer, answers, tokenizer, eval_dataset=val_dataset)['f1_mean']
            np.save('/content/drive/MyDrive/QA_training/results/text_prep_results/f1_for_different_models_and_with_no_answ_dev',
                f1_mean)



Step 1 of 10.


Step 2 of 10.
Step 2 of 10.


Step 3 of 10.
Step 3 of 10.


Step 4 of 10.
Step 4 of 10.


Step 5 of 10.
Step 5 of 10.


Step 6 of 10.


In [ ]:
df = pd.DataFrame(data=f1_mean, columns=["answer", "no_answer"], index=model_names)
df

,answer,no_answer
bert_1_text,0.659507,0.721278
bert_2_texts,0.651718,0.637679
bert_3_texts,0.660258,0.566190
bert_2_random_texts,0.671546,0.716232
bert_3_rendom_texts,0.656065,0.682759
